# DynamoDB 

## Crear una tabla para guardar los tweets

El primer paso para insertar datos en DynamoDB es la creación de una tabla donde almacenar los datos. Esta operación Se puede hacer de las tres maneras de interactuar con AWS, pues en esta práctica lo haremos de forma programatica utilizando el SDK de python: [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/quickstart.html). 



### Crear un cliente para DynamoDB

Las siguientes líneas crean un client para DynamoDB 

In [1]:
import boto3

# Initialize the DynamoDB resource
dynamodb = boto3.resource("dynamodb", region_name="us-east-1")
dynamodb_client = boto3.client("dynamodb", region_name="us-east-1")

`import boto3`: Imports the Boto3 library, which is the Amazon Web Services (AWS) SDK for Python. We can use services like Amazon S3 and Amazon EC2.

`boto3.resource()`: Provides a higher-level abstraction and is typically used for interacting with DynamoDB tables in an object-oriented way. It's often used for simple CRUD (Create, Read, Update, Delete) operations.

`boto3.client()`: Provides a lower-level abstraction and is typically used for more complex operations, such as batch operations or when you need more fine-grained control over the requests being sent to DynamoDB.

## Crear tablas para DynamoDB

Cada tabla de DynamoDB tiene que proporcionar un nombre y un esquema para las claves de la BB.DD (*Partition Key*, y *Hash Key* si es necesaria). La sintaxis del método `create_table` necesita varios parámetros: 
- el `nombre de la tabla`
- el `esquema para la claves`
- el `tipo de claves`
- el `throughput esperado para esta tabla`

### Crear una tabla para tweets per idioma y timestamp 

Usaríamos este código si crearamos las tablas manualmente. Pero en nuestro caso haremos importación desde S3, para cargar un gran volument de datos, así que este código nos sirve solo como referencia. Podemos probarlo, pero 

In [2]:
table_name = "TestTable" 
table = dynamodb.create_table(
    TableName=table_name,
    # KEY SCHEMA
    KeySchema=[
        {"AttributeName": "user", "KeyType": "HASH"},  # Partition Key (HASH)
        {"AttributeName": "timestamp", "KeyType": "RANGE"},  # Sort Key (RANGE)
    ], 
    # DEFINE SCHEMA
    AttributeDefinitions=[
        {"AttributeName": "user", "AttributeType": "S"}, # USER IS A STRING
        {"AttributeName": "timestamp", "AttributeType": "N"} # TIMESTAMP IS A NUMBER 
    ], # AN ATTRIBUTE OF AN ELEMENT CAN BE A LIST
    # WE CAN TO WRITE 5 UNITS AND READ 5. RELATED TO PROVISIONED CAPACITY
    ProvisionedThroughput={"ReadCapacityUnits": 5, "WriteCapacityUnits": 5},
)

# Wait for the table to be created
table.meta.client.get_waiter("table_exists").wait(TableName=table_name)
print(f"Table '{table_name}' created successfully!")

Table 'TestTable' created successfully!


In [3]:
#Delete the table
dynamodb_client.delete_table(TableName=table_name)

{'TableDescription': {'TableName': 'TestTable',
  'TableStatus': 'DELETING',
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 5,
   'WriteCapacityUnits': 5},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-east-1:842912048264:table/TestTable',
  'TableId': '2ed1f792-0665-4fc7-a298-f07dd5bde7c7',
  'DeletionProtectionEnabled': False},
 'ResponseMetadata': {'RequestId': 'UA8HSRI9PJR00MMBRTJI7KKPTRVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 12 Feb 2025 18:53:08 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '354',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'UA8HSRI9PJR00MMBRTJI7KKPTRVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2975200431'},
  'RetryAttempts': 0}}

## Modelizar tweets

Vamos a enriqueces el model de Tweet que ya conocemos extrayendo también el `timestamp` (el tiempo de creación exacto del tweet). 

Añadimos también un método (`to_dynamo_json`) que devuelve el mismo Tweet como `Item` de DynamoDB en formato JSON: esto significa que Dynamo puede leer JSON (en un formato especifico para Dynamo) y "cargarlos" en la BB.DD.

In [4]:
from dataclasses import dataclass
import json 

@dataclass
class Tweet:
    """Class to model a Tweet"""
    id: int         # The unique ID of a tweet
    content: str    # The textual content of a tweet
    author: str     # The nickname of the author of the tweet
    language: str   # The language of the tweet
    timestamp: int  # The timestamp of the tweet, in epoch

    def to_dynamo_json_by_lang(self):
        tweet = {
            "Item": {
                "language": {
                    "S": self.language
                },
                "timestamp_tweetid": {
                    "S": f'{self.timestamp}#{self.id}'
                },
                "content": {
                    "S": self.content
                }
            }
        }
        return json.dumps(tweet)
    
    def to_dynamo_json_by_user(self):
        tweet = {
            "Item": {
                "user": {
                    "S": self.author
                },
                "timestamp": {
                    "N": f"{self.timestamp}"
                },
                "content": {
                    "S": self.content
                },
                "language": {
                    "S": self.language
                }
            }
        }
        return json.dumps(tweet)
        

This script defines a Python class named Tweet using the @dataclass decorator. The Tweet class is designed to model a tweet with its associated attributes.

Attributes:
- id: a unique integer identifier for the tweet
- content: the textual content of the tweet as a string
- author: the nickname of the author of the tweet as a string
- language: the language of the tweet as a string
- timestamp: the timestamp of the tweet in epoch time (the number of seconds that have elapsed since January 1, 1970) as an integer

Methods:
The Tweet class has two methods:
- `to_dynamo_json_by_lang`: This method returns a JSON string representing the tweet in a format suitable for Amazon DynamoDB. The primary key is a composite key consisting of the language and a concatenation of the timestamp and tweet ID.
- `to_dynamo_json_by_user`: This method returns a JSON string representing the tweet in a format suitable for Amazon DynamoDB. The primary key is the author (user), and the sort key is the timestamp.


### Ejemplo de Tweet y de su formato JSON para DynamoDB

In [5]:
tweet = Tweet(123, "esto es el contenido de un Tweet", "luca", "es", 1234567890)
print(tweet.to_dynamo_json_by_lang())
print(tweet.to_dynamo_json_by_user())

{"Item": {"language": {"S": "es"}, "timestamp_tweetid": {"S": "1234567890#123"}, "content": {"S": "esto es el contenido de un Tweet"}}}
{"Item": {"user": {"S": "luca"}, "timestamp": {"N": "1234567890"}, "content": {"S": "esto es el contenido de un Tweet"}, "language": {"S": "es"}}}


## Cargar tweets a Dynamo DB

No podemos insertar un Tweet a la vez en Dynamo: insertar 1M de tweets tardaría horas! Esto porque Dynamo no está hecho para procesar un enorme volumen de tráfico en un tiempo limitado (*batch*), sino - similarmente a una BB.DD. convencional - un tráfico constante a lo largo del tiempo (*real-time*). 

Por lo tanto usaremos una solución alternativa. Para poder cargar un volumen de datos interesante, utilizaremos la opción de cargar una tabla desde S3. Esto se traduce en que, utilizando Spark, leeremos unos datos de input y los transformaremos en unos datos de output que almacenaremos en S3 y que se podrán cargar en DynamoDB, respetando el formato de representación de datos de entrada de DynamoDB. 

El proceso será: 

- (Notebook) Usar spark para leer datos de input, y transformalos en datos de output aptos para la carga. Tendremos que escribir estos datos en S3. 
- (Consola Web de AWS) Usar la funcionalidad de Dynamo de cargar desde S3 para indicar nuestro directorio de output como fuente de datos para una tabla. 

Este proceso se tendrá que **realizar una sola vez**, y será nuestra manera de *simular un volumen de datos entrantes* en DynamoDB

Para poder alcanzar este objectivo con éxito, necesitaremos superar un escollo con que hasta la fecha no hemos trabajado: **las credenciales de AWS**. 

In [6]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import sagemaker_pyspark
import os 

# os.environ['AWS_PROFILE'] = "default"

conf = SparkConf()
conf.set("spark.driver.extraClassPath", ":".join(sagemaker_pyspark.classpath_jars()))
# USE OUR BUCKET AND WRITE ON OUR OWN BUCKET
conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "com.amazonaws.auth.DefaultAWSCredentialsProviderChain")

spark = (
    SparkSession
    .builder
    .config(conf=conf)
    .appName("test")
    .getOrCreate()
)
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/02/12 19:05:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Set up a SparkSession with AWS credentials and SageMaker PySpark configuration. 
Importing Libraries
The script starts by importing the necessary libraries:
- `pyspark`: the Python API for Apache Spark
- `sagemaker_pyspark`: a library that integrates SageMaker with PySpark
- `os`: a library for interacting with the operating system

Setting AWS Credentials
The script sets the AWS_PROFILE environment variable to "default". This tells the AWS SDK to use the default credentials profile, which is typically stored in the `~/.aws/credentials` file.

Configuring Spark
- The script creates a `SparkConf` object and sets two configuration properties:

-   `spark.driver.extraClassPath`: this property is set to the list of JAR files required by SageMaker PySpark, which are obtained using sagemaker_pyspark.classpath_jars(). The ":".join() method is used to concatenate the list of JAR files into a single string, separated by colons.

-   `spark.hadoop.fs.s3a.aws.credentials.provider`: this property is set to com.amazonaws.auth.DefaultAWSCredentialsProviderChain, which tells Spark to use the default AWS credentials provider chain to authenticate with S3.

Creating a SparkSession
The script creates a SparkSession using the SparkSession.builder API. The config method is used to pass the SparkConf object created earlier, and the appName method is used to set the name of the Spark application to "test". Finally, the getOrCreate method is used to create the SparkSession if it doesn't already exist.

Getting the SparkContext
The script gets the `SparkContext` object from the `SparkSession` using the `sparkContext` attribute. The `SparkContext` is the core entry point for `Spark functionality`.


In [9]:
import json

def toTweet(line: str):
      try:
        parsed = json.loads(line)
        epoch = int(parsed['timestamp_ms'])
        ## NOTA: usando ['user']['screen_name']
        return Tweet(parsed['id'], parsed['text'], parsed['user']['screen_name'], parsed['lang'], epoch)
      except Exception as e:
        return None
    
# ONLY READING 1 FILE
rdd = sc.textFile('s3a://mudab-2025-big-data/twitter-data/Eurovision-00.json')
processed = (rdd  # rdd[string]
    .map(toTweet) # parse string into Tweet. Return rdd[Tweet]
    .filter(lambda x: x is not None) # filter empty values. Return same type (rdd[Tweet])
)
rdd.take(1)
processed.take(1)

[Tweet(id=995443356309311493, content='RT @jk_rowling: France ❤️ #Eurovision', author='_hoodstery', language='fr', timestamp=1526167165244)]

In [11]:
# ELEMENTS TO INSERT INTO A TABLE IN DYNOMO BD
dynamo_by_lang = processed.map(lambda x: x.to_dynamo_json_by_lang()) # RDD[TWEET].  
dynamo_by_user = processed.map(lambda x: x.to_dynamo_json_by_user()) # RDD[]

dynamo_by_lang.saveAsTextFile('s3a://mudab-2025-pc1262057/twitter-data-for-dynamo/by_lang/1/')
dynamo_by_user.saveAsTextFile('s3a://mudab-2025-pc1262057/twitter-data-for-dynamo/by_user/1/') 

The resulting RDDs, `dynamo_by_lang` and `dynamo_by_user`, contain JSON strings that can be written to Amazon DynamoDB.

El siguiente calculo proporciona los timestamp a los extremos de nuestra colección. Son **valores en milisegundos**. 

In [12]:
max_ts = processed.map(lambda x: x.timestamp).reduce(lambda x, y: max(x, y))
min_ts = processed.map(lambda x: x.timestamp).reduce(lambda x, y: min(x, y))

print(f'Lower ts: {min_ts}, higher ts: {max_ts}')

Lower ts: 1526167165244, higher ts: 1526169318161


## Cargar datos en Dynamo

Una vez procesados los Tweets, los podemos cargar en Dynamo DB de manera batch a través de la consola de S3. Este proceso tardará unos minutos, y lo realizaremos a través de la interfaz Web de amazon 

## Query Dynamo 

Antes de comenzar, tenemos que tener dos tablas después de cargar datos via S3 

### Tabla de Usuarios por Tiempo 

Tiene el siguiente aspecto: 
- la primary key es el nombre de usuario: 'username'
- la sort key es el `timestamp` del Tweet

### Tabla de Idiomas por Tiempo 

Tiene el siguiente aspecto: 
- la primary key es el idioma: 'language'
- la sort key es el `timestamp + el ID` del Tweet: dos campos se suelen separar por el caracter `#`, de forma que un valor aparecerà come `123#321`

In [13]:
tweet_by_user_table = "TweetByUser"
tweet_by_language_table = "TweetByLang"

### Últimos tweets por usuario 

Un caso de uso típico para DynamoDb podría ser por ejemplo de visualizar los últimos N tweets de un usuario especifico. En este caso, con el ID de usuario como *Partition Key*, y el timestamp como *Sort Key* será muy sencillo realizar una query. 

In [14]:
import boto3
from boto3.dynamodb.conditions import Key

table = dynamodb.Table(tweet_by_user_table)

def last_10_tweets_for_user(user: str): 
    # Query for the last 10 tweets for a specific user
    response = table.query(
        KeyConditionExpression=Key('user').eq(user), # OUR USER MUST BE THE SAME AS THE VALUE WE PASSED
        ScanIndexForward=False,  # Descending order: latest tweets first
        Limit=10
    )
    # Process the response
    tweets = response.get('Items', [])
    return tweets

for tweet in last_10_tweets_for_user('celia22975'):
    print(tweet)

{'content': 'RT @IgualdadLGBT: Tenemos un nuevo referente lésbico, Saara Aalto, cantante de Finlandia #Eurovision #FinalEurovision https://t.co/zEDD3dUj…', 'user': 'celia22975', 'language': 'es', 'timestamp': Decimal('1526169312928')}
{'content': 'RT @playz: #Eurovision #FinalEurovision ¡Se acabó! Así ha terminado la jornada de @Ricky_ot2017 en el Instagram de Playz. Ahora, ¡todos de…', 'user': 'celia22975', 'language': 'es', 'timestamp': Decimal('1526169255068')}
{'content': 'RT @ManelNMusic: Oye Israel, que esto lo hice yo antes que tu... #Eurovision #FinalEurovision https://t.co/bBnDjB9s4z', 'user': 'celia22975', 'language': 'es', 'timestamp': Decimal('1526169228713')}
{'content': "RT @eurovision_tve: 🔴 CRÓNICA Israel y el baile de la 'gallina' ganan #Eurovision en el año en el que Amaia y Alfred enamoraron a España ht…", 'user': 'celia22975', 'language': 'es', 'timestamp': Decimal('1526169225734')}
{'content': 'RT @VodafoneTV_es: Israel ha jugado con fuego pero consigue llevarse #E

#### Query para todos los tweets en un idioma dentro de un interval de tiempo 

In [15]:
from datetime import datetime 
def get_tweets_in_time_range(language, start_time, end_time):
    """Fetches tweets within a specific time range."""
    table = dynamodb.Table(tweet_by_language_table)

    start_key = f"{start_time}#"
    end_key = f"{end_time}#"

    response = table.query(
        KeyConditionExpression="#lang = :val AND timestamp_tweetid BETWEEN :start AND :end",
        ExpressionAttributeNames={"#lang": "language"},  # Alias for reserved keyword
        ExpressionAttributeValues={":val": language, ":start": start_key, ":end": end_key},
        ScanIndexForward=True,  # True = Oldest first
    )
    return response.get("Items", [])

start = min_ts
recent_tweets = get_tweets_in_time_range("en", start, start + 60000)
print(len(recent_tweets))
for tweet in recent_tweets[0:10]:
    print(tweet)

865
{'content': 'RT @britneyxcheetos: you mean Chub-Bi https://t.co/KijWpZxRYR', 'language': 'en', 'timestamp_tweetid': '1526167165271#995443356422557696'}
{'content': 'RT @snugglycamila: i’m not surprised eurovision is confusing for americans since the concept of the person with the most votes actually win…', 'language': 'en', 'timestamp_tweetid': '1526167165367#995443356825202690'}
{'content': 'RT @gi_de_off: #Eurovision \ni just adopted two boys 🇺🇦❤️🇨🇿 https://t.co/2ZSvphRPxV', 'language': 'en', 'timestamp_tweetid': '1526167165419#995443357043298304'}
{'content': 'hey #EUROVISION next year i am thinking of participating in eurovision song contest as spongebob squerapants... wha… https://t.co/PR24YGk0Hk', 'language': 'en', 'timestamp_tweetid': '1526167165466#995443357240393728'}
{'content': 'RT @davidmackau: my fav #eurovision tradition is the international BuzzFeed accounts coming together to roast the shit out of @BuzzFeedUK h…', 'language': 'en', 'timestamp_tweetid': '152616716550

This script is designed to fetch tweets within a specific time range from a DynamoDB table. Here's a breakdown of how it works:
The script starts by importing the datetime module, which is not used in this snippet, but might be used elsewhere in the code to handle date and time operations.

The get_tweets_in_time_range function takes three parameters:
- language: the language of the tweets to be fetched
- start_time: the start of the time range
- end_time: the end of the time range

The function uses the dynamodb library to interact with a DynamoDB table. The table name is stored in the tweet_by_language_table variable, which is not defined in this snippet.
The start_key and end_key variables are created by concatenating the start_time and end_time with a "#" symbol. This is likely done to create a unique key for the DynamoDB query.

The table.query method is used to fetch tweets from the DynamoDB table. The query uses the following parameters:
KeyConditionExpression: specifies the conditions for the query. In this case, it's looking for tweets where the language attribute matches the :val parameter, and the timestamp_tweetid attribute is between the :start and :end parameters.

ExpressionAttributeNames: provides an alias for the language attribute, which is a reserved keyword in DynamoDB.

ExpressionAttributeValues: provides the values for the :val, :start, and :end parameters.
ScanIndexForward: specifies the order in which the results are returned. In this case, it's set to True, which means the results are returned in ascending order (i.e., oldest tweets first).

The function returns a list of tweets that match the query conditions. If no tweets are found, an empty list is returned.

The script then calls the get_tweets_in_time_range function with the following parameters:
- language: "en" (English)
- start_time: min_ts (which is not defined in this snippet)
- end_time: min_ts + 60000 (which is 1 minute after min_ts)

The script prints the number of tweets returned by the function, and then prints the first 10 tweets in the list.

### Serie temporal por idioma 

Podemos crear una serie temporal para visualizar el número de tweets por idioma en intervalos consecutivos de tiempo. Para obtener el dato bruto, podemos hacer una secuencia de queries a Dynamo. 


In [16]:
import time
from boto3.dynamodb.conditions import Key
import boto3

# Initialize the DynamoDB resource and table
dynamodb = boto3.resource("dynamodb", region_name="us-east-1")
table = dynamodb.Table(tweet_by_language_table)

def count_tweets_for_language_in_interval(language, start_ts, end_ts):
    """
    Count tweets for a given language between start_ts and end_ts.
    """
    # Construct the lower and upper bounds of the sort key.
    # Note: This assumes that the sort key is stored as "timestamp#tweet_id"
    start_key = f"{start_ts}#"
    end_key = f"{end_ts}#"    
    response = table.query(
        KeyConditionExpression=Key('language').eq(language) & 
                               Key('timestamp_tweetid').between(start_key, end_key),
        Select='COUNT'
    )
    return response.get('Count', 0)

def to_readable_date(timestamp_ms: str): 
    utc_date = datetime.utcfromtimestamp(timestamp_ms/1000)
    formatted_utc = utc_date.strftime('%Y-%m-%d %H:%M:%S')
    return formatted_utc

def timeseries(lang: str, time_from: int, time_to: int, delta: int):
    """
    Count tweets in a specific language and interval, with a fixed step.
    """
    values = []
    current_ts = time_from
    while(current_ts < time_to):
        tweet_count = count_tweets_for_language_in_interval(lang, current_ts, current_ts + delta)
        values.append((to_readable_date(current_ts), tweet_count))
        current_ts += delta
    return values

start_time = min_ts - (min_ts % 60000)
one_minute = 60 * 1000

en_ts_1m = timeseries('en', start_time, max_ts, one_minute)
es_ts_5m = timeseries('es', start_time, max_ts, 5 * one_minute)


print(en_ts_1m)
print(es_ts_5m)

[('2018-05-12 23:19:00', 488), ('2018-05-12 23:20:00', 884), ('2018-05-12 23:21:00', 880), ('2018-05-12 23:22:00', 912), ('2018-05-12 23:23:00', 886), ('2018-05-12 23:24:00', 887), ('2018-05-12 23:25:00', 924), ('2018-05-12 23:26:00', 876), ('2018-05-12 23:27:00', 877), ('2018-05-12 23:28:00', 887), ('2018-05-12 23:29:00', 849), ('2018-05-12 23:30:00', 872), ('2018-05-12 23:31:00', 880), ('2018-05-12 23:32:00', 840), ('2018-05-12 23:33:00', 842), ('2018-05-12 23:34:00', 880), ('2018-05-12 23:35:00', 846), ('2018-05-12 23:36:00', 866), ('2018-05-12 23:37:00', 862), ('2018-05-12 23:38:00', 852), ('2018-05-12 23:39:00', 845), ('2018-05-12 23:40:00', 764), ('2018-05-12 23:41:00', 809), ('2018-05-12 23:42:00', 841), ('2018-05-12 23:43:00', 855), ('2018-05-12 23:44:00', 832), ('2018-05-12 23:45:00', 850), ('2018-05-12 23:46:00', 832), ('2018-05-12 23:47:00', 881), ('2018-05-12 23:48:00', 800), ('2018-05-12 23:49:00', 829), ('2018-05-12 23:50:00', 818), ('2018-05-12 23:51:00', 817), ('2018-05